# CX2Network: Access node, edge, and network attributes

In [ ]:
from ndex2.cx2 import CX2Network, RawCX2NetworkFactory
from ndex2 import constants
import ndex2.client as nc
import ndex2
import networkx as nx
import pandas as pd
import os
import json

In [ ]:
# EXAMPLE NETWORK FROM NDEx

uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303'

# Create NDEx2 python client
client = ndex2.client.Ndex2(host='ndexbio.org')

# Create CX2Network factory
factory = RawCX2NetworkFactory()

# Get network as cx2 stream
client_resp = client.get_network_as_cx2_stream(uuid)

# Convert downloaded network to CX2Network object
cx2_network = factory.get_cx2network(json.loads(client_resp.content))

### Nodes

Loop through all **nodes** and print when we reach the RAD52 node

In [ ]:
for node_id, node in cx2_network.get_nodes().items():
    if node.get('v')['name'] == 'RAD52':
        print(node)
        break

Nodes and edges are indexed by id (@id from the example above).  To look up a specific node by it's _name_ you can create a reverse look up index as follows

In [ ]:
#============================
# CREATE A NAME to ID LOOKUP 
#============================
node_name_lookup = {node.get('v').get('name', None): node_id for node_id, node in cx2_network.get_nodes().items()}

#========================
# GET THE 'MAP2K1' NODE
#========================
map2k1_node_id = node_name_lookup.get('MAP2K1')
map2k1_node = cx2_network.get_node( map2k1_node_id )
print(map2k1_node)

#========================================================
# GET THE 'MAP2K1' NODE (USING METHOD FROM CX2NETWORK)
#========================================================
map2k1_node_id = cx2_network.lookup_node_id_by_name( 'MAP2K1' )
map2k1_node = cx2_network.get_node(map2k1_node_id)

### Node attributes

Get the node attribute (Pathway) for MAP2K1

In [ ]:
map2k1_pathway_attribute = cx2_network.get_node(map2k1_node_id).get(constants.ASPECT_VALUES).get('Pathway')
print(map2k1_pathway_attribute)

### Edges

Print all the **edges** where MAP2K1 is either a source or target

In [ ]:
map2k1_edges = []

for edge_id, edge in cx2_network.get_edges().items():
    if edge.get('s') == map2k1_node_id or edge.get('t') == map2k1_node_id:
        map2k1_edges.append(edge.get('id'))

        print(edge)


Or, if you would like to see the node labels represented in these edges you can look up and print the node name

In [ ]:
for edge_id, edge in cx2_network.get_edges().items():
    if edge.get('s') == map2k1_node_id or edge.get('t') == map2k1_node_id:
        source_node = cx2_network.get_node( edge.get('s') )
        target_node = cx2_network.get_node( edge.get('t') )
        print('Source: %s Target: %s Interaction: %s' % (source_node.get(constants.ASPECT_VALUES).get('name'), target_node.get(constants.ASPECT_VALUES).get('name'), edge.get(constants.ASPECT_VALUES).get('interaction')))

### Edge attributes

Print the **edge attributes** (yeastSscore) for MAP2K1 edges

In [ ]:
for edge_id in map2k1_edges:
    print(cx2_network.get_edge(edge_id).get(constants.ASPECT_VALUES).get('yeastSscore'))


### Network attributes

Print the reference attribute for this network

In [ ]:
cx2_network.get_network_attributes().get('reference')

Or print the html content in the value field

In [ ]:
from IPython.display import display, HTML

display(HTML(cx2_network.get_network_attributes().get('reference')))